### Simple notebook to summarize top wind turbine manufacturers and models used in USA

Intent is to use as a list in other work to enforce turbine mfr/model naming convention

In [ ]:
import sys
import os
import glob
from pathlib import Path

cwd = Path.cwd()
print('Current working directory: ', cwd)

print('python version: ', sys.version) # uses python version 3.11.5

import polars as pl
print('polars version: ', pl.__version__) # uses polars version 1.17.1

import requests # to get data from the website https://eerscmap.usgs.gov/uswtdb/data/
import zipfile # to extract the zip file downloaded from the website



In [ ]:

# URL of the zip file
url = "https://eerscmap.usgs.gov/uswtdb/assets/data/uswtdbCSV.zip"

try:
    # Download the file with a timeout
    response = requests.get(url, timeout=10)
    response.raise_for_status()  # Check if the request was successful

    # Save the file in the current directory
    with open(cwd / "uswtdbCSV.zip", "wb") as file:
        file.write(response.content)

    print("File downloaded and saved as uswtdbCSV.zip")

except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")



In [ ]:
# unzip file uswtdbCSV.zip to the current directory
import zipfile
with zipfile.ZipFile(cwd / 'uswtdbCSV.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:


# get the name of the csv file in the unzipped folder
file = glob.glob(str(cwd / 'uswtdb*.csv'))[0]
print(file)

In [ ]:
# get data from csv file as polars df
data = pl.read_csv(file, null_values=['', 'NA', 'null'])
print(data.head())

In [ ]:
# print the columns in the df
for i,c in enumerate(data.columns):
    print(c, end=',')
    if i%10 == 0:
        print('')

In [ ]:
# group data by t_manu and get counts to identify popularity by mfr
mfrs = (data.group_by(['t_manu']).agg(pl.col('eia_id').count().alias('mfr_count'))
            .sort('mfr_count', descending=True)
            .filter( (pl.col('mfr_count') > 500) & (pl.col('t_manu').is_not_null()) ))
mfrs

In [ ]:
# get the main models (hh separately as there are several models with multiple hh)
models = (data.with_columns(pl.col('t_model').str.replace(' ', '').alias('t_model'))
              .group_by(['t_manu', 't_model', 't_cap','t_rd'])
              .agg(pl.col('eia_id').count().alias('model_count'))
              .sort('model_count', descending=True)
              .filter(pl.col('model_count') > 100))
models

In [ ]:
# get hubheights for the models as a list - cast to string so they can be joined into a single string value to export to csv
hh = (data.filter( (pl.col('t_hh').is_not_null()) & (pl.col('t_manu').is_not_null()) & (pl.col('t_model').is_not_null()) )
          .group_by(['t_manu', 't_model', 't_cap','t_rd'])
          .agg(pl.col('t_hh').unique().sort().cast(pl.Utf8))
          .with_columns(pl.col('t_hh').list.join(', ').alias('t_hh'))
          .sort('t_manu', 't_cap', 't_rd'))
hh


In [ ]:
# join models and hh on t_manu, t_model, t_cap, t_rd
joined = (models.join(hh, on=['t_manu', 't_model', 't_cap', 't_rd'], how='inner')
          .select(['t_manu', 't_model', 't_cap', 't_rd','t_hh','model_count'])
          .sort(['t_manu', 't_model']))

# join mfrs to set popluarity order by count
joined = mfrs.join(joined, on=['t_manu'], how='left').sort(['mfr_count','t_cap','t_rd'], descending=[True,False,False])
joined

# on 2024-12-14, the filters applied above just happened to result in a top 100 list.


In [ ]:
joined.write_csv('wind_turbines.csv')